<a href="https://colab.research.google.com/github/Nukles/analytics-componentized-patterns/blob/master/Stock_Analysis_v3_(my_own).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on: https://towardsdatascience.com/modeling-your-stock-portfolio-performance-with-python-fbba4ef2ef11


**Import encessary packages**

In [ ]:
from google.colab import files
import io
import numpy as np
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import yfinance as yf
!pip install -q pandas_market_calendars
import pandas_market_calendars as mcal
from plotly.offline import init_notebook_mode, plot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.1/190.1 kB 7.1 MB/s eta 0:00:00


**Asset Details**

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1eHqdhmtBnSRKKMNgfbG699KYl8keHlA-e-CeysSFAys/edit#gid=1588171313').worksheet("Asset Details")

assetDetails_df = pd.DataFrame(worksheet.get_all_records())
assetDetails_df.sample(n=10).sort_index()

,Symbol,In Open Positions,Google Ticker,Investment Strategy,Sector,SA Sector,Industry,Type,Ind-Type,Currency,...,PE,Expense Ratio,EPS,30d perf,30d perf vs SPY,30d perf vs Ref Index,Index,Reference Index,Yahoo Ticker,Added
4,ROKU,N,NASDAQ:ROKU,,Communication Services,Communication Services,Movies and Entertainment,Stock,Communication Services Stock,USD,...,,,-3.62,- 8.48 %,- 10.86 ppt,- 9.85 ppt,NASDAQ,NASDAQ:QQQ,ROKU,01/01/2018
32,NOVN,N,SWX:NOVN,,Health Care,Health Care,Pharmaceuticals,Stock,Health Care Stock,CHF,...,32.71,,2.81,+ 11.73 %,+ 9.36 ppt,+ 4.72 ppt,SWX,LON:VWCG,NOVN,01/01/2018
35,INO,N,NASDAQ:INO,,Health Care,Health Care,Biotechnology,Stock,Health Care Stock,USD,...,,,-1.17,- 10.99 %,- 13.36 ppt,- 12.36 ppt,NASDAQ,NASDAQ:QQQ,INO,01/01/2018
42,ZM,N,NASDAQ:ZM,,Information Technology,Information Technology,Application Software,Stock,Information Technology Stock,USD,...,185.11,,0.34,- 7.89 %,- 10.26 ppt,- 9.26 ppt,NASDAQ,NASDAQ:QQQ,ZM,24/08/2022
56,HHS,N,NASDAQ:HHS,,Communication Services,Communication Services,Advertising,Stock,Communication Services Stock,USD,...,,,-1.42,- 10.81 %,- 13.18 ppt,- 12.18 ppt,NASDAQ,NASDAQ:QQQ,HHS,23/10/2022
63,RINF,Y,NYSEARCA:RINF,Inflation Protection,Inflation Protection,,Nontraditional Bond,ETF,Inflation Protection ETF,USD,...,,,,+ 1.26 %,- 1.11 ppt,- 1.11 ppt,NYSEARCA,NYSEARCA:SPY,RINF,23/10/2022
64,TBF,N,NYSEARCA:TBF,Inflation Protection,Inflation Protection,,Trading-Inverse Debt,ETF,Inflation Protection ETF,USD,...,,,,- 2.32 %,- 4.70 ppt,- 4.70 ppt,NYSEARCA,NYSEARCA:SPY,TBF,23/10/2022
87,NXPI,N,NASDAQ:NXPI,Dividend Consistency,Information Technology,Information Technology,Semiconductors,Stock,Information Technology Stock,USD,...,,,,- 5.39 %,- 7.76 ppt,- 6.76 ppt,NASDAQ,NASDAQ:QQQ,NXPI,07/11/2022
96,ASC,Y,NYSE:ASC,Value/Growth,Energy,Energy,Oil and Gas Storage and Transportation,Stock,Energy Stock,USD,...,,,,- 1.59 %,- 3.96 ppt,- 3.96 ppt,NYSE,NYSEARCA:SPY,ASC,25/10/2022
110,DXJ,Y,NYSEARCA:DXJ,Value/Growth,International Equity,International Equity,Japan Stock,ETF,International Equity ETF,USD,...,,,,+ 3.73 %,+ 1.36 ppt,+ 1.36 ppt,NYSEARCA,NYSEARCA:SPY,DXJ,25/10/2022


**Activity Statement**

In [ ]:
uploaded = files.upload()
filename_activitystatement = next(iter(uploaded))

Saving U1966785_20220408_20230407 (2) (Activity Statement).csv to U1966785_20220408_20230407 (2) (Activity Statement).csv


In [ ]:
# Import file into DataFrame
activitystatement_df = pd.read_csv(io.BytesIO(uploaded[filename_activitystatement]),skiprows=6)
activitystatement_df = activitystatement_df.loc[activitystatement_df['DataDiscriminator'].isin(["Trade"])]
activitystatement_df = activitystatement_df.loc[activitystatement_df['Asset Category'].isin(["Stocks - Held with Interactive Brokers (U.K.) Limited carried by Interactive Brokers LLC"])]

activitystatement_df = activitystatement_df[["Symbol","Currency","Date/Time","Quantity","Comm/Fee"]]

df = activitystatement_df

# Yahoo Ticker and Name info
#df = pd.merge(activitystatement_df, assetDetails_df[['Symbol','Yahoo Ticker','Name','Type']], on='Symbol')


# Select only Trades
#df = df.loc[df['DataDiscriminator'].isin(["Trade"])]
#df = df.loc[df['Asset Category'].isin(["Stocks - Held with Interactive Brokers (U.K.) Limited carried by Interactive Brokers LLC"])]

# Convert Quantity to int
df['Quantity'] = df['Quantity'].astype('int')
#df['Basis'] = df['Basis'].astype('float')

# Add Index Column and Move it to the First one
df = df.reset_index(drop=True)
df['Index'] = df.index
temp_cols=df.columns.tolist()
new_cols=temp_cols[-1:] + temp_cols[:-1]
df=df[new_cols]

# Add "Security" Column
#df['Security'] = df['Name']

# Add "Symbol" Column from "Yahoo Ticker"
#df = df.drop('Symbol', axis=1)
#df['Symbol'] = df['Yahoo Ticker']

# Add "Qty" Column baased on "Quantity"
#df['Qty'] = df['Quantity'].abs()

# Add "Type" Column (Buy or Sell) based on "Quantity"
#df['Order'] = np.where(df['Quantity']>0,'Buy','Sell')

# Add "Open date" Column based on "Date/Time"
#df["Open date"] = pd.to_datetime(df["Date/Time"])

# Convert "Date/Time" to datetime
df["Date/Time"] = pd.to_datetime(df["Date/Time"])


# Add "Adj cost" Column based on "Basis"
#df["Adj cost"] = df["Basis"]

# Add "Adj cost per share" Column
#df["Adj cost per share"] = df["Adj cost"]/df["Qty"]

# Keep only some columns
df.reset_index()
#activitystatement_df = df[['Yahoo Ticker','Type','Currency','Date/Time','Quantity','Order']].sort_values(by=['Date/Time'])
#activitystatement_df = activitystatement_df.rename(columns={"Yahoo Ticker": "Symbol"})
activitystatement_df = df

activitystatement_df

# If you need to find anything - df[df.eq("7270.T").any(1)]


,Index,Symbol,Currency,Date/Time,Quantity,Comm/Fee
0,0,HELN,CHF,2022-07-05 08:16:25,-3,-5
1,1,HELN,CHF,2022-12-15 08:33:02,-13,-5
2,2,NOVN,CHF,2022-08-12 03:03:27,-3,-5
3,3,NOVN,CHF,2022-12-16 03:24:48,-10,-5
4,4,AIRd,EUR,2022-06-23 03:22:03,-9,-1.25
...,...,...,...,...,...,...
123,123,VT,USD,2022-12-13 09:30:02,90,0
124,124,VYM,USD,2022-07-14 07:21:51,-5,-1.01203359
125,125,VYM,USD,2022-12-13 12:49:53,-31,-1.0828289
126,126,WMT,USD,2022-07-25 16:17:10,-5,-1.0149625


# Open Positions


In [ ]:
# Import Open Positions as of 1. Jan 2022

uploaded = files.upload()
filename_openpositions = next(iter(uploaded))

Saving U1966785_20220103 (Open Positions).csv to U1966785_20220103 (Open Positions) (2).csv


In [ ]:
openpositions_df = pd.read_csv(io.BytesIO(uploaded[filename_openpositions]),skiprows=6,names=range(16))
openpositions_df.columns = openpositions_df.iloc[0]
openpositions_df = openpositions_df[1:]
openpositions_df = openpositions_df[openpositions_df['Open Positions']=='Open Positions']
openpositions_df = openpositions_df[openpositions_df['Header']=='Data']

openpositions_df.reset_index()
openpositions_df = openpositions_df[["Symbol","Quantity","Close Price","Currency"]].dropna().sort_values(by=['Symbol'])
#openpositions_df = openpositions_df.

openpositions_df


,Symbol,Quantity,Close Price,Currency
10,AAPL,36,182.01,USD
5,AIRd,31,116.14,EUR
1,AMD,47,27,CHF
11,AMD,50,150.24,USD
12,AMZN,3,3408.09,USD
6,B7A,33,38.86,EUR
13,BA,17,207.86,USD
14,BOTZ,181,36.09,USD
15,BXMT,56,31.11,USD
16,CL,13,84.59,USD


# Newer Code v2

For each date in the calendar, sum the Buys and subtract the Sells until the selected date from Activity Statement (and ideally in the future, forget about Open Positions)



In [ ]:
activitystatement_df

In [ ]:
def create_market_cal(start, end):
    nyse = mcal.get_calendar('NYSE')
    schedule = nyse.schedule(stocks_start, stocks_end)
    market_cal = mcal.date_range(schedule, frequency='1D')
    market_cal = market_cal.tz_localize(None)
    market_cal = [i.replace(hour=0) for i in market_cal]
    return market_cal

stocks_start = datetime.datetime(2022, 1, 1)
stocks_end = datetime.datetime(2022, 12, 31)

market_cal = create_market_cal(stocks_start, stocks_end)

In [ ]:
# Create function to make snapshots

def snapshot(df, date):
  snapshot = df.loc[df['Date/Time'] < date].groupby('Symbol')['Quantity'].sum().reset_index() # This sums all the trades (Buy and Sell) up to the date provided
  snapshot_df = pd.DataFrame(snapshot)
  snapshot_df['Snapshot Date']=date
  snapshot_df = pd.merge(snapshot_df, assetDetails_df, on='Symbol')[['Snapshot Date','Symbol','Quantity','Currency','Type','Yahoo Ticker']]
  return snapshot_df



In [ ]:
# Create dictionary of Dataframes for each day

snapshots = {}

for date in market_cal:
  snapshots[date]=snapshot(activitystatement_df, date)

snapshots[market_cal[150]]

,Snapshot Date,Symbol,Quantity,Currency,Type,Yahoo Ticker
0,2022-08-09,AAPL,-11,USD,Stock,AAPL
1,2022-08-09,AIRd,-9,EUR,Stock,AIR.DE
2,2022-08-09,AMD,-5,USD,Stock,AMD
3,2022-08-09,AMZN,-18,USD,Stock,AMZN
4,2022-08-09,AVGO,4,USD,Stock,AVGO
5,2022-08-09,BA,-7,USD,Stock,BA
6,2022-08-09,BOTZ,-181,USD,ETF,BOTZ
7,2022-08-09,BXMT,-11,USD,Stock,BXMT
8,2022-08-09,CL,-4,USD,Stock,CL
9,2022-08-09,COST,-2,USD,Stock,COST


In [ ]:
activitystatement_df.sort_values('Date/Time')

,Index,Symbol,Currency,Date/Time,Quantity,Comm/Fee
71,71,JETS,USD,2022-06-15 10:07:50,-188,-1.0987047
85,85,NFLX,USD,2022-06-16 10:35:56,-18,-1.073617624
58,58,IBUY,USD,2022-06-16 10:37:50,-47,-1.050162959
28,28,BOTZ,USD,2022-06-17 10:21:07,-181,-1.108790593
90,90,ONLN,USD,2022-06-17 10:23:27,-69,-1.059549001
...,...,...,...,...,...,...
11,11,IES,EUR,2022-12-16 03:00:15,713,-3
12,12,LHAd,EUR,2022-12-16 03:00:28,175,-3
7,7,DTEd,EUR,2022-12-16 03:00:29,68,-3
3,3,NOVN,CHF,2022-12-16 03:24:48,-10,-5


In [ ]:
market_cal[0]

Timestamp('2022-09-01 00:00:00')

In [ ]:
activitystatement_df

#Check for duplicates

In [ ]:

def show_duplicates(df,column):
  df = df[column][df[column].duplicated()]
  return df

show_duplicates(assetDetails_df,'Symbol')


#snapshot_df['Symbol'][snapshot_df['Symbol'].duplicated()]

Series([], Name: Symbol, dtype: object)

In [ ]:
df = pd.DataFrame([[1, 9, 10, 12],[2, 3, 1, 2],[3, 4, 5, 2],[4, 2, 1, 1]], columns=['A', 'B','C','D'])
df

,A,B,C,D
0,1,9,10,12
1,2,3,1,2
2,3,4,5,2
3,4,2,1,1


In [ ]:
df

In [ ]:
df = pd.DataFrame([['A', 201901, 10, 201801, 201801], 
                   ['B', 201902, 11, 201801, 201802], 
                   ['B', 201903, 13, 201801, 201803],
                   ['B', 201905, 18, 201801, 201805],
                   ['A', 201906, 80, 201801, 201806],
                   ['A', 202001, 10, 201901, 201901], 
                   ['A', 202002, 11, 201901, 201902], 
                   ['A', 202003, 13, 201901, 201903],
                   ['A', 202004, 18, 201901, 201904],
                   ['B', 202005, 80, 201901, 201905],
                   ['A', 202006, 80, 201901, 201906],
                   ['B', 201901, 10, 201801, 201801], 
                   ['A', 201902, 11, 201801, 201802], 
                   ['A', 201903, 13, 201801, 201803],
                   ['A', 201905, 18, 201801, 201805],
                   ['B', 201906, 80, 201801, 201806],
                   ['B', 202001, 10, 201901, 201901], 
                   ['B', 202002, 11, 201901, 201902], 
                   ['B', 202003, 13, 201901, 201903],
                   ['B', 202004, 18, 201901, 201904],
                   ['A', 202005 ,80, 201901, 201905],
                   ['B', 202006 ,80, 201901, 201906]],
                  columns = ['Store','yearweek','sales','Start_PY','PY'])
df[df['Store'] == 'B'].groupby(['yearweek'])['sales'].sum()


yearweek
201901    10
201902    11
201903    13
201905    18
201906    80
202001    10
202002    11
202003    13
202004    18
202005    80
202006    80
Name: sales, dtype: int64

In [ ]:
technologies   = ({
    'Courses':["Spark","PySpark","Hadoop","Python","Pandas","Hadoop","Spark","Python"],
    'Fee' :[22000,25000,23000,24000,26000,25000,25000,22000],
    'Duration':['30days','50days','55days','40days','60days','35days','55days','50days'],
    'Discount':[1000,2300,1000,1200,2500,1300,1400,1600]
                })
df = pd.DataFrame(technologies, columns=['Courses','Fee','Duration','Discount'])
print(df)


   Courses    Fee Duration  Discount
0    Spark  22000   30days      1000
1  PySpark  25000   50days      2300
2   Hadoop  23000   55days      1000
3   Python  24000   40days      1200
4   Pandas  26000   60days      2500
5   Hadoop  25000   35days      1300
6    Spark  25000   55days      1400
7   Python  22000   50days      1600


In [ ]:
df2 = df.groupby('Courses').sum()
print(df2)


           Fee  Discount
Courses                 
Hadoop   48000      2300
Pandas   26000      2500
PySpark  25000      2300
Python   46000      2800
Spark    47000      2400


<ipython-input-30-67171a88dec3>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [ ]:
df['Sales_PY'] = (df.apply(lambda r: df.loc[(df['yearweek'] >= r['Start_PY'])
                                           &(df['yearweek'] <= r['PY']) 
                                           &(df['Store']==r['Store']) ,'sales'].sum(),axis=1)
                )
df

,Store,yearweek,sales,Start_PY,PY,Sales_PY
0,A,201901,10,201801,201801,0
1,B,201902,11,201801,201802,0
2,B,201903,13,201801,201803,0
3,B,201905,18,201801,201805,0
4,A,201906,80,201801,201806,0
5,A,202001,10,201901,201901,10
6,A,202002,11,201901,201902,21
7,A,202003,13,201901,201903,34
8,A,202004,18,201901,201904,34
9,B,202005,80,201901,201905,52


# Newer Code v1

In [ ]:
# Create Market Cal

def create_market_cal(start, end):
    nyse = mcal.get_calendar('NYSE')
    schedule = nyse.schedule(stocks_start, stocks_end)
    market_cal = mcal.date_range(schedule, frequency='1D')
    market_cal = market_cal.tz_localize(None)
    market_cal = [i.replace(hour=0) for i in market_cal]
    return market_cal

stocks_start = datetime.datetime(2022, 1, 1)
stocks_end = datetime.datetime(2022, 12, 31)

market_cal = create_market_cal(stocks_start, stocks_end)

In [ ]:
activitystatement_df.sample(n=3)

,Symbol,Type,Currency,Date/Time,Quantity,Order
25,BA,Stock,USD,2022-12-09 12:42:05,-1,Sell
34,CECO,Stock,USD,2022-12-13 09:34:47,25,Buy
125,VYM,ETF,USD,2022-12-13 12:49:53,-31,Sell


In [ ]:
# Create daily balance (Step 3)

def fifo(daily_positions, sales, date):
    sales = sales[sales['Date/Time'] == date]
    daily_positions = daily_positions[daily_positions['Date/Time'] <= date]
    positions_no_change = daily_positions[~daily_positions['Symbol'].isin(sales['Symbol'].unique())]
    adj_positions = pd.DataFrame()
    for sale in sales.iterrows():
        adj_positions = adj_positions.append(position_adjust(daily_positions, sale))
    adj_positions = adj_positions.append(positions_no_change)
    adj_positions = adj_positions[adj_positions['Quantity'] > 0]
    return adj_positions


def time_fill(portfolio, market_cal):
    sales = portfolio[portfolio['Order'] == 'Sell'].groupby(['Symbol','Date/Time'])['Quantity'].sum() # 
    sales = sales.reset_index()
    per_day_balance = []
    for date in market_cal:
        if (sales['Date/Time'] == date).any():
            portfolio = fifo(portfolio, sales, date)
        daily_positions = portfolio[portfolio['Date/Time'] <= date]
        daily_positions = daily_positions[daily_positions['Order'] == 'Buy']
        daily_positions['Date Snapshot'] = date
        per_day_balance.append(daily_positions)
    return per_day_balance


positions_per_day = time_fill(activitystatement_df, market_cal)
positions_per_day

# Older Code

In [ ]:
# Create table of stocks daily (for now only the traded ones, in the future will have to be also the starting ones)

stocks_start = datetime.datetime(2022, 1, 1)
stocks_end = datetime.datetime(2022, 12, 31)

def create_market_cal(start, end):
    nyse = mcal.get_calendar('NYSE')
    schedule = nyse.schedule(stocks_start, stocks_end)
    market_cal = mcal.date_range(schedule, frequency='1D')
    market_cal = market_cal.tz_localize(None)
    market_cal = [i.replace(hour=0) for i in market_cal]
    return market_cal

market_cal = create_market_cal(stocks_start,stocks_end)
#market_cal

def get_data(stocks, start, end):
    def data(ticker):
        df = yf.download(ticker, start=start, end=(end + datetime.timedelta(days=1)))
        df['symbol'] = ticker
        df.index = pd.to_datetime(df.index)
        return df
    datas = map(data, stocks)
    return(pd.concat(datas, keys=stocks, names=['Ticker', 'Date'], sort=True))

symbols = activitystatement_df["Yahoo Ticker"].unique()
#get_data('META',stocks_start,stocks_end)
dailyclose_stocks = get_data(symbols, stocks_start, stocks_end)


In [ ]:
dailyclose_stocks

Adj Close       Close        High         Low  \
Ticker  Date                                                         
HELN.SW 2022-01-03  104.420540  109.300003  109.300003  108.099998   
        2022-01-04  107.191071  112.199997  112.300003  110.000000   
        2022-01-05  107.477676  112.500000  112.900002  111.099998   
        2022-01-06  107.477676  112.500000  113.000000  110.599998   
        2022-01-07  109.101784  114.199997  114.199997  112.699997   
...                        ...         ...         ...         ...   
WMT     2022-12-23  143.183151  143.770004  143.800003  142.279999   
        2022-12-27  143.222977  143.809998  144.449997  143.190002   
        2022-12-28  140.713257  141.289993  143.669998  140.910004   
        2022-12-29  141.569748  142.149994  143.020004  141.750000   
        2022-12-30  141.211227  141.789993  141.990005  140.809998   

                          Open   Volume   symbol  
Ticker  Date                                      
HELN.SW 2022-01-03  108.800003    63285  HELN.SW  
        2022-01-04  110.000000   124116  HELN.SW  
        2022-01-05  112.500000    92493  HELN.SW  
        2022-01-06  111.800003    89070  HELN.SW  
        2022-01-07  113.000000   103533  HELN.SW  
...                        ...      ...      ...  
WMT     2022-12-23  143.050003  3182500      WMT  
        2022-12-27  144.039993  3348200      WMT  
        2022-12-28  143.470001  5082100      WMT  
        2022-12-29  142.139999  3057300      WMT  
        2022-12-30  141.559998  3835300      WMT  

[15588 rows x 7 columns]

In [ ]:
# Create table of daily Exchange Rates for Currencies

worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1eHqdhmtBnSRKKMNgfbG699KYl8keHlA-e-CeysSFAys/edit#gid=1588171313').worksheet("Exchange Rates")

exchangerates_df = pd.DataFrame(worksheet.get_all_records())
#exchangerates_df
currencies = exchangerates_df["Currency"].unique()
dailyclose_currencies = get_data(currencies, stocks_start, stocks_end)
dailyclose_currencies['symbol']=dailyclose_currencies['symbol'].str[:3]
dailyclose_currencies

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Adj Close     Close      High       Low      Open  \
Ticker   Date                                                            
JPYCHF=X 2022-01-03   0.007920  0.007920  0.007977  0.007910  0.007921   
         2022-01-04   0.007963  0.007963  0.007965  0.007869  0.007963   
         2022-01-05   0.007886  0.007886  0.007914  0.007877  0.007887   
         2022-01-06   0.007900  0.007900  0.007951  0.007891  0.007900   
         2022-01-07   0.007952  0.007952  0.007970  0.007934  0.007952   
...                        ...       ...       ...       ...       ...   
USDCHF=X 2022-12-26   0.932880  0.932880  0.933900  0.930900  0.932880   
         2022-12-27   0.932010  0.932010  0.932090  0.926900  0.932010   
         2022-12-28   0.929200  0.929200  0.930600  0.924640  0.929200   
         2022-12-29   0.927710  0.927710  0.928720  0.921190  0.927710   
         2022-12-30   0.922850  0.922850  0.926110  0.920320  0.922850   

                     Volume symbol  
Ticker   Date                       
JPYCHF=X 2022-01-03       0    JPY  
         2022-01-04       0    JPY  
         2022-01-05       0    JPY  
         2022-01-06       0    JPY  
         2022-01-07       0    JPY  
...                     ...    ...  
USDCHF=X 2022-12-26       0    USD  
         2022-12-27       0    USD  
         2022-12-28       0    USD  
         2022-12-29       0    USD  
         2022-12-30       0    USD  

[780 rows x 7 columns]

In [ ]:
def position_adjust(daily_positions, sale):
    stocks_with_sales = pd.DataFrame()
    buys_before_start = daily_positions[daily_positions['Order'] == 'Buy'].sort_values(by='Date/Time')
    for position in buys_before_start[buys_before_start['Symbol'] == sale[1]['Symbol']].iterrows():
        if position[1]['Qty'] <= sale[1]['Qty']:
            sale[1]['Qty'] -= position[1]['Qty']
            position[1]['Qty'] = 0
        else:
            position[1]['Qty'] -= sale[1]['Qty']
            sale[1]['Qty'] -= sale[1]['Qty']
        stocks_with_sales = stocks_with_sales.append(position[1])
    return stocks_with_sales


def portfolio_start_balance(portfolio, start_date):
    positions_before_start = portfolio[portfolio['Date/Time'] <= start_date]
    future_positions = portfolio[portfolio['Date/Time'] >= start_date]
    sales = positions_before_start[positions_before_start['Date/Time'] == 'Sell'].groupby(['Symbol'])['Qty'].sum()
    sales = sales.reset_index()
    positions_no_change = positions_before_start[~positions_before_start['Symbol'].isin(sales['Symbol'].unique())]
    adj_positions_df = pd.DataFrame()
    for sale in sales.iterrows():
        adj_positions = position_adjust(positions_before_start, sale)
        adj_positions_df = adj_positions_df.append(adj_positions)
    adj_positions_df = adj_positions_df.append(positions_no_change)
    adj_positions_df = adj_positions_df.append(future_positions)
    adj_positions_df = adj_positions_df[adj_positions_df['Qty'] > 0]
    return adj_positions_df

active_portfolio = portfolio_start_balance(activitystatement_df, stocks_start)

KeyError: ignored

In [ ]:
def create_market_cal(start, end):
    nyse = mcal.get_calendar('NYSE')
    schedule = nyse.schedule(stocks_start, stocks_end)
    market_cal = mcal.date_range(schedule, frequency='1D')
    market_cal = market_cal.tz_localize(None)
    market_cal = [i.replace(hour=0) for i in market_cal]
    return market_cal


def get_data(stocks, start, end):
    def data(ticker):
        df = yf.download(ticker, start=start, end=(end + datetime.timedelta(days=1)))
        df['symbol'] = ticker
        df.index = pd.to_datetime(df.index)
        return df
    datas = map(data, stocks)
    return(pd.concat(datas, keys=stocks, names=['Ticker', 'Date'], sort=True))


def get_benchmark(benchmark, start, end):
    benchmark = get_data(benchmark, start, end)
    benchmark = benchmark.drop(['symbol'], axis=1)
    benchmark.reset_index(inplace=True)
    return benchmark

###### Step 2

def position_adjust(daily_positions, sale):
    stocks_with_sales = pd.DataFrame()
    buys_before_start = daily_positions[daily_positions['Type'] == 'Buy'].sort_values(by='Open date')
    for position in buys_before_start[buys_before_start['Symbol'] == sale[1]['Symbol']].iterrows():
        if position[1]['Qty'] <= sale[1]['Qty']:
            sale[1]['Qty'] -= position[1]['Qty']
            position[1]['Qty'] = 0
        else:
            position[1]['Qty'] -= sale[1]['Qty']
            sale[1]['Qty'] -= sale[1]['Qty']
        stocks_with_sales = stocks_with_sales.append(position[1])
    return stocks_with_sales


def portfolio_start_balance(portfolio, start_date):
    positions_before_start = portfolio[portfolio['Open date'] <= start_date]
    future_positions = portfolio[portfolio['Open date'] >= start_date]
    sales = positions_before_start[positions_before_start['Type'] == 'Sell.FIFO'].groupby(['Symbol'])['Qty'].sum()
    sales = sales.reset_index()
    positions_no_change = positions_before_start[~positions_before_start['Symbol'].isin(sales['Symbol'].unique())]
    adj_positions_df = pd.DataFrame()
    for sale in sales.iterrows():
        adj_positions = position_adjust(positions_before_start, sale)
        adj_positions_df = adj_positions_df.append(adj_positions)
    adj_positions_df = adj_positions_df.append(positions_no_change)
    adj_positions_df = adj_positions_df.append(future_positions)
    adj_positions_df = adj_positions_df[adj_positions_df['Qty'] > 0]
    return adj_positions_df

######

def fifo(daily_positions, sales, date):
    sales = sales[sales['Open date'] == date]
    daily_positions = daily_positions[daily_positions['Open date'] <= date]
    positions_no_change = daily_positions[~daily_positions['Symbol'].isin(sales['Symbol'].unique())]
    adj_positions = pd.DataFrame()
    for sale in sales.iterrows():
        adj_positions = adj_positions.append(position_adjust(daily_positions, sale))
    adj_positions = adj_positions.append(positions_no_change)
    adj_positions = adj_positions[adj_positions['Qty'] > 0]
    return adj_positions


def time_fill(portfolio, market_cal):
    sales = portfolio[portfolio['Type'] == 'Sell.FIFO'].groupby(['Symbol','Open date'])['Qty'].sum()
    sales = sales.reset_index()
    per_day_balance = []
    for date in market_cal:
        if (sales['Open date'] == date).any():
            portfolio = fifo(portfolio, sales, date)
        daily_positions = portfolio[portfolio['Open date'] <= date]
        daily_positions = daily_positions[daily_positions['Type'] == 'Buy']
        daily_positions['Date Snapshot'] = date
        per_day_balance.append(daily_positions)
    return per_day_balance


# matches prices of each asset to open date, then adjusts for  cps of dates
def modified_cost_per_share(portfolio, adj_close, start_date):
    df = pd.merge(portfolio, adj_close, left_on=['Date Snapshot', 'Symbol'],
                  right_on=['Date', 'Ticker'], how='left')
    df.rename(columns={'Close': 'Symbol Adj Close'}, inplace=True)
    df['Adj cost daily'] = df['Symbol Adj Close'] * df['Qty']
    df = df.drop(['Ticker', 'Date'], axis=1)
    return df


# merge portfolio data with latest benchmark data and create several calcs
def benchmark_portfolio_calcs(portfolio, benchmark):
    portfolio = pd.merge(portfolio, benchmark, left_on=['Date Snapshot'],
                         right_on=['Date'], how='left')
    portfolio = portfolio.drop(['Date'], axis=1)
    portfolio.rename(columns={'Close': 'Benchmark Close'}, inplace=True)
    benchmark_max = benchmark[benchmark['Date'] == benchmark['Date'].max()]
    portfolio['Benchmark End Date Close'] = portfolio.apply(lambda x: benchmark_max['Close'], axis=1)
    benchmark_min = benchmark[benchmark['Date'] == benchmark['Date'].min()]
    portfolio['Benchmark Start Date Close'] = portfolio.apply(lambda x: benchmark_min['Close'], axis=1)
    return portfolio


def portfolio_end_of_year_stats(portfolio, adj_close_end):
    adj_close_end = adj_close_end[adj_close_end['Date'] == adj_close_end['Date'].max()]
    portfolio_end_data = pd.merge(portfolio, adj_close_end, left_on='Symbol',
                                  right_on='Ticker')
    portfolio_end_data.rename(columns={'Close': 'Ticker End Date Close'}, inplace=True)
    portfolio_end_data = portfolio_end_data.drop(['Ticker', 'Date'], axis=1)
    return portfolio_end_data


# Merge the overall dataframe with the adj close start of year dataframe for YTD tracking of tickers.
def portfolio_start_of_year_stats(portfolio, adj_close_start):
    adj_close_start = adj_close_start[adj_close_start['Date'] == adj_close_start['Date'].min()]
    portfolio_start = pd.merge(portfolio, adj_close_start[['Ticker', 'Close', 'Date']],
                                    left_on='Symbol', right_on='Ticker')
    portfolio_start.rename(columns={'Close': 'Ticker Start Date Close'}, inplace=True)
    portfolio_start['Adj cost per share'] = np.where(portfolio_start['Open date'] <= portfolio_start['Date'],
                                                          portfolio_start['Ticker Start Date Close'],
                                                          portfolio_start['Adj cost per share'])
    portfolio_start['Adj cost'] = portfolio_start['Adj cost per share'] * portfolio_start['Qty']
    portfolio_start = portfolio_start.drop(['Ticker', 'Date'], axis=1)
    portfolio_start['Equiv Benchmark Shares'] = portfolio_start['Adj cost'] / portfolio_start['Benchmark Start Date Close']
    portfolio_start['Benchmark Start Date Cost'] = portfolio_start['Equiv Benchmark Shares'] * portfolio_start['Benchmark Start Date Close']
    return portfolio_start


def calc_returns(portfolio):
    portfolio['Benchmark Return'] = portfolio['Benchmark Close'] / portfolio['Benchmark Start Date Close'] - 1
    portfolio['Ticker Return'] = portfolio['Symbol Adj Close'] / portfolio['Adj cost per share'] - 1
    portfolio['Ticker Share Value'] = portfolio['Qty'] * portfolio['Symbol Adj Close']
    portfolio['Benchmark Share Value'] = portfolio['Equiv Benchmark Shares'] * portfolio['Benchmark Close']
    portfolio['Stock Gain / (Loss)'] = portfolio['Ticker Share Value'] - portfolio['Adj cost']
    portfolio['Benchmark Gain / (Loss)'] = portfolio['Benchmark Share Value'] - portfolio['Adj cost']
    portfolio['Abs Value Compare'] = portfolio['Ticker Share Value'] - portfolio['Benchmark Start Date Cost']
    portfolio['Abs Value Return'] = portfolio['Abs Value Compare']/portfolio['Benchmark Start Date Cost']
    portfolio['Abs. Return Compare'] = portfolio['Ticker Return'] - portfolio['Benchmark Return']
    return portfolio


def per_day_portfolio_calcs(per_day_holdings, daily_benchmark, daily_adj_close, stocks_start):
    df = pd.concat(per_day_holdings, sort=True)
    mcps = modified_cost_per_share(df, daily_adj_close, stocks_start)
    bpc = benchmark_portfolio_calcs(mcps, daily_benchmark)
    pes = portfolio_end_of_year_stats(bpc, daily_adj_close)
    pss = portfolio_start_of_year_stats(pes, daily_adj_close)
    returns = calc_returns(pss)
    return returns


def line_facets(df, val_1, val_2):
    grouped_metrics = combined_df.groupby(['Symbol','Date Snapshot'])[[val_1,val_2]].sum().reset_index()
    grouped_metrics = pd.melt(grouped_metrics, id_vars=['Symbol','Date Snapshot'],
                              value_vars=[val_1, val_2])
    fig = px.line(grouped_metrics, x="Date Snapshot", y="value",
                  color='variable', facet_col="Symbol", facet_col_wrap=5)
    #plot(fig)
    fig.show()


def line(df, val_1, val_2):
    grouped_metrics = combined_df.groupby(['Date Snapshot'])[[val_1,val_2]].sum().reset_index()
    grouped_metrics = pd.melt(grouped_metrics, id_vars=['Date Snapshot'],
                              value_vars=[val_1, val_2])
    fig = px.line(grouped_metrics, x="Date Snapshot", y="value", 
                  color='variable')
    #plot(fig)
    fig.show()


In [ ]:
portfolio_df = activityreport_df
portfolio_df['Open date'] = pd.to_datetime(portfolio_df['Open date'])
symbols = portfolio_df.Symbol.unique()

stocks_start = datetime.datetime(2021, 1, 1)
stocks_end = datetime.datetime(2022, 12, 31)

daily_adj_close = get_data(symbols, stocks_start, stocks_end)

daily_adj_close = daily_adj_close[['Close']].reset_index()
daily_benchmark = get_benchmark(['SPY'], stocks_start, stocks_end)
daily_benchmark = daily_benchmark[['Date', 'Close']]



<ipython-input-47-1ee5c77283b2>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:

market_cal = create_market_cal(stocks_start, stocks_end)
active_portfolio = portfolio_start_balance(portfolio_df, stocks_start)
positions_per_day = time_fill(active_portfolio, market_cal)
combined_df = per_day_portfolio_calcs(positions_per_day, daily_benchmark,
                                      daily_adj_close, stocks_start)



<ipython-input-46-48c1c0919335>:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-46-48c1c0919335>:52: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [ ]:
line_facets(combined_df, 'Ticker Return', 'Benchmark Return')
line(combined_df, 'Stock Gain / (Loss)', 'Benchmark Gain / (Loss)')